# Q1 2026 forecast scenarios

In [1]:
import sys  # no installation needed
from pathlib import Path  # no installation needed

ROOT = Path(r"C:\\Users\\quantbase\\Desktop\\ecom_forecast")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [2]:
import pandas as pd  # already in env - no new install
import numpy as np  # already in env - no new install

from src.config import ProjectPaths  # no installation needed
from src.forecast import attainability_check, derive_q4_baselines, run_q1_forecast  # no installation needed


In [3]:
paths = ProjectPaths.from_root()
paths.ensure_directories()
forecast_dir = paths.outputs_dir / 'forecast'
forecast_dir.mkdir(parents=True, exist_ok=True)
assumptions = paths.load_assumptions()


In [4]:
driver = pd.read_pickle(paths.outputs_dir / 'drivers' / 'driver_matrix.pkl')
driver = driver.sort_values('Day').reset_index(drop=True)
epsilon = float(assumptions.get('epsilon', 1e-9))
if 'return_rate_gross' not in driver.columns:
    driver['return_rate_gross'] = driver['returns_abs'] / (driver['Gross sales'] + epsilon)
if 'discount_rate_gross' not in driver.columns:
    driver['discount_rate_gross'] = (-driver['Discounts']) / (driver['Gross sales'] + epsilon)


In [5]:
baselines = derive_q4_baselines(driver, assumptions)
forecast_long = run_q1_forecast(driver, assumptions)
attainability = attainability_check(forecast_long, target_cm_pct=0.15)
forecast_long


,scenario,month,paid_sessions,organic_sessions,total_sessions,conversion_rate,aov_gross,gross_sales,returns_abs,discounts_abs,...,shipping,merchant_fees,cogs,ad_spend,mer,cm$,cm%,return_rate_gross,discount_rate_gross,weighted_cogs_pct
0,conservative,Jan,148282.368132,46006.670769,194289.038901,0.015735,225.964680,6.908204e+05,115563.454815,10859.179637,...,25986.244027,14109.943905,235491.246524,203043.186813,2.779693,85767.134916,0.151962,0.167284,0.015719,0.417243
1,conservative,Feb,133932.461538,41554.412308,175486.873846,0.015735,225.964680,6.239668e+05,104379.894671,9808.291285,...,23471.446218,12744.465462,212701.771054,183393.846154,2.779693,77467.089601,0.151962,0.167284,0.015719,0.417243
2,conservative,Mar,148282.368132,46006.670769,194289.038901,0.015735,225.964680,6.908204e+05,115563.454815,10859.179637,...,25986.244027,14109.943905,235491.246524,203043.186813,2.779693,85767.134916,0.151962,0.167284,0.015719,0.417243
3,base_return_fix,Jan,156086.703297,47884.494066,203971.197363,0.017735,228.247152,8.256840e+05,133995.608798,12153.450653,...,30748.747316,16988.373493,280813.338271,213729.670330,3.179413,137254.810296,0.201983,0.162284,0.014719,0.413243
4,base_return_fix,Feb,140981.538462,43250.510769,184232.049231,0.017735,228.247152,7.457791e+05,121028.291818,10977.310267,...,27773.062092,15344.337348,253637.853922,193046.153846,3.179413,123972.086719,0.201983,0.162284,0.014719,0.413243
5,base_return_fix,Mar,156086.703297,47884.494066,203971.197363,0.017735,228.247152,8.256840e+05,133995.608798,12153.450653,...,30748.747316,16988.373493,280813.338271,213729.670330,3.179413,137254.810296,0.201983,0.162284,0.014719,0.413243
6,aggressive,Jan,171695.373626,49292.861538,220988.235165,0.020735,232.812095,1.066807e+06,178460.185354,16769.412215,...,38949.273213,21789.442074,361916.892452,235102.637363,3.707222,213819.437858,0.245325,0.167284,0.015719,0.415243
7,aggressive,Feb,155079.692308,44522.584615,199602.276923,0.020735,232.812095,9.635679e+05,161189.844836,15146.565872,...,35179.988709,19680.786389,326892.677053,212350.769231,3.707222,193127.234194,0.245325,0.167284,0.015719,0.415243
8,aggressive,Mar,171695.373626,49292.861538,220988.235165,0.020735,232.812095,1.066807e+06,178460.185354,16769.412215,...,38949.273213,21789.442074,361916.892452,235102.637363,3.707222,213819.437858,0.245325,0.167284,0.015719,0.415243


In [6]:
metrics_for_pivot = ['net_sales', 'cm$', 'cm%']
pivot = (
    forecast_long.melt(id_vars=['scenario', 'month'], var_name='metric', value_name='value')
    .query('metric in @metrics_for_pivot')
    .pivot_table(index=['month', 'metric'], columns='scenario', values='value', aggfunc='first')
)
pivot = pivot.reset_index()


In [7]:
def flatten_dict(d, prefix=''):
    items = []
    for k, v in d.items():
        new_key = f'{prefix}.{k}' if prefix else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key))
        else:
            items.append((new_key, v))
    return items
assumption_items = flatten_dict(assumptions.get('forecast_q1', {}), prefix='forecast_q1')
assumptions_snapshot = pd.DataFrame(assumption_items, columns=['key', 'value'])


In [8]:
forecast_long.to_csv(forecast_dir / 'q1_forecast_long.csv', index=False)
pivot.to_csv(forecast_dir / 'q1_forecast_pivot.csv', index=False)
attainability.to_csv(forecast_dir / 'attainability.csv', index=False)
assumptions_snapshot.to_csv(forecast_dir / 'assumptions_snapshot.csv', index=False)
print('Baselines:', baselines)
print('Base return fix scenario:')
print(forecast_long.query('scenario == "base_return_fix"'))
print('Attainability:')
print(attainability)
['q1_forecast_long.csv', 'q1_forecast_pivot.csv', 'attainability.csv', 'assumptions_snapshot.csv']


Baselines: {'sessions_total_daily_avg': 6549.428571428572, 'paid_sessions_daily_avg': 5035.054945054945, 'organic_sessions_daily_avg': 1514.3736263736264, 'conversion_rate_avg': 0.017735345632424675, 'aov_gross_avg': 228.24715168411404, 'return_rate_gross_avg': 0.1672843714240882, 'discount_rate_gross_avg': 0.015719251753756765, 'ad_spend_daily_avg': 6894.505494505494, 'weighted_cogs_pct': 0.4152434137861537}
Base return fix scenario:
          scenario month  paid_sessions  organic_sessions  total_sessions  \
3  base_return_fix   Jan  156086.703297      47884.494066   203971.197363   
4  base_return_fix   Feb  140981.538462      43250.510769   184232.049231   
5  base_return_fix   Mar  156086.703297      47884.494066   203971.197363   

   conversion_rate   aov_gross    gross_sales    returns_abs  discounts_abs  \
3         0.017735  228.247152  825683.999156  133995.608798   12153.450653   
4         0.017735  228.247152  745779.096012  121028.291818   10977.310267   
5         0.017

['q1_forecast_long.csv',
 'q1_forecast_pivot.csv',
 'attainability.csv',
 'assumptions_snapshot.csv']